# Segmentation using the qim.register module
*TODO: Figure our how to "install" the module so that I don't need to put the github folder in the path.*

## Load modules

In [1]:
import sys, os
#sys.path.insert(0, '/home/hektor/scripts')
sys.path.insert(0, '/lunarc/nobackup/projects/qim')

import qim.registration as registration
import qim.tools as tools
import tifffile
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

from skimage import filters


## Read data
The example data is from a multimodal imaging experiment (x-rays and neutron) at ILL. The sample is a rat bone with an implant. This is part of Elin Törnqvists PhD research.

In [2]:
qimfolder = '/lunarc/nobackup/projects/qim/QIM_2020_LU_Elin_Bio'
datapath = os.path.join(qimfolder,'pvt21/reg-common-peaks_new')
x_data = os.path.join(datapath,'x_registered.tiff')
n_data = os.path.join(datapath,'n-bin1.tif')

x_volume = tifffile.imread(x_data)
n_volume = tifffile.imread(n_data)


## Create the dual histogram

This will create a dual (2D) histogram where the intensity of the x-ray data is on the x-axis and the intensity of the neutron data is on the y-axis. The number of bin in the histogram 
The x-ray data will be on the x-axis and the neutron data on the y-axis. The number of bins will be the same on both axes.

*TODO: perhaps it is useful to have different number of bins on the axes*


In [4]:
nbins = 256
H,xedges,yedges = registration.make_dual_histogram(x_volume,n_volume,nbins)

registration.plot_dual_histogram(H,xedges,yedges,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=2500)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Fit Gaussians to the histogram
Now we will fit 2D Gaussians to regions of the histogram. There is an automatic peak searching feature (which doesn't work very well). There is also the possibility to provide inital guesses on the position of the Gaussians. This is done by giving a list of coordinates to the keyword argument `marker` of `registration.setup_markers`. The coordinates are specified like this:
```
ml = [[x-ray 1, neutron 1], [x-ray 2, neutron 2], ..., [x-ray N, neutron N]]
```
The intensities can be read from the histogram above.

The size of the region to fit each Gaussian to can be controlled by the `fitdist` keyword to `registration.fit_gaussians`.

*TODO: Improve automatic peak search*


In [5]:
# generate markers
#ml = [[0,2500],[8000,0],[65400,8700]]
#ml = [[0,2500],[7175, 3139], [20800, 46000], [25800, 15900], [48000, 28000], [66085, 16200]]#x reg onto n
ml = [[20800, 46000], [35000, 30000], [65400, 16200]]#x reg onto n
markers = registration.setup_markers(H,xedges,yedges,markers=ml)
# Fit a Gaussian to each marker
opts = registration.fit_gaussians(H,markers,xedges,yedges,fitdist=20)
# Make a plot with markers and Gaussians
registration.plot_dual_histogram(H,xedges,yedges,xlabel='X-ray intensity',ylabel='Neutron intensity',vmax=1500,markers=markers,fits=opts)

Trying to fit peak at (20863.7, 46207.3)
Initial guess: [20863.682 46207.295 100.0 100.0 0.0 1462.000000 0.0]
Fitted parameters [19855.249 45649.577 3106.0 2460.2 -1.8 1845.996288 -2.0]
Fitted 1 peaks
Trying to fit peak at (35199.5, 30079.5)
Initial guess: [35199.463 30079.541 100.0 100.0 0.0 3271.000000 0.0]
Fitted parameters [36630.630 28768.173 3078.6 2397.3 -0.6 6268.389957 -3.6]
Fitted 2 peaks
Trying to fit peak at (65407.0, 16255.8)
Marker is on an edge
Initial guess: [65407.002 16255.752 100.0 100.0 0.0 112817.000000 0.0]
Fitted parameters [65452.993 15306.796 60.3 1567.5 0.0 198735.402787 5.4]
Fitted 3 peaks


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Build phase diagram
We will now do a labelling of the histogram based on the fitted Gaussians. Basically the idea is to assign a number to each of the Gaussians and then color each pixel in the histogram with the number of the closest Gaussian.

To find the closest distance we use the [Mahalanobis distance function](https://en.wikipedia.org/wiki/Mahalanobis_distance).

We will also try to compute a maximum distance for assigning pixels to phases, such that at least 90% of the parts of the histogram >0 is assigned a label. This step can be skipped using the `sigma` keyword of `registration.phase_diagram`.

For the example data we are not interested in the lower left quadrant of the histogram (this contains mostly background and artefacts), we therefore use a low number of `sigma` to limit the size of the phases.

*TODO: Check how the distance function works*

*TODO: Fix the color mapping*

In [8]:
coverage = 0.9
phasediagram = registration.phase_diagram(H,xedges,yedges,opts,coverage=coverage,sigma=4)
nphases=phasediagram.max()

#plot the phasediagram
cmap = 'Set3' #vill bara ha x färger
Y,X = np.meshgrid(yedges,xedges,indexing='ij') #coordinates in dual histogram
fig=plt.figure()
m=plt.pcolormesh(X,Y,phasediagram,cmap=cmap)
plt.xlabel('X-ray intensity')
plt.ylabel('Neutron intensity')
cb=fig.colorbar(m,ticks=range(nphases+1))
m.set_clim(0.5, nphases + 0.5)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Map from the phase diagram to the volume
We will now color the volume based on the labels in the phase diagram. If `slicenr = None` the full volume will be colored, otherwise just the one slice corresponding to `slicenr` will be colored. When coloring the full volume you can use multiple processors by giving a number to the `nprocs` keyword of `registration.map_to_volume`. 

**WARNING: Do not run this with multiple processors on the log-in nodes of Aurora**. 


In [9]:
%%time
slicenr = None
labels = registration.map_to_volume(x_volume,n_volume,xedges,yedges,phasediagram,slicenr=slicenr,nprocs=20)

Setting up pool with 20 processors
CPU times: user 1.51 s, sys: 3.77 s, total: 5.29 s
Wall time: 43.1 s


## Plot slices of the labelled volume

In [11]:
if not slicenr:
    z,x,y = labels.shape[0]//2, labels.shape[1]//2,labels.shape[2]//2
    fig,axes=plt.subplots(1,3,figsize=(12,6))
    ax = axes.ravel()
    fig.suptitle('Labelled slices')
    m=ax[0].imshow(labels[z],cmap=cmap,vmin=1)
    ax[1].imshow(labels[:,x,:],cmap=cmap,vmin=1)
    ax[2].imshow(labels[:,:,y],cmap=cmap,vmin=1)
    cb=fig.colorbar(m,ticks=range(nphases+1))
    m.set_clim(0.5, nphases + 0.5)

    fig,axes=plt.subplots(1,3,figsize=(12,6))
    ax = axes.ravel()
    fig.suptitle('X-ray data')
    ax[0].imshow(x_volume[z],cmap='Greys_r')
    ax[1].imshow(x_volume[:,x,:],cmap='Greys_r')
    ax[2].imshow(x_volume[:,:,y],cmap='Greys_r')

    fig,axes=plt.subplots(1,3,figsize=(12,6))
    ax = axes.ravel()
    fig.suptitle('Neutron data')
    ax[0].imshow(n_volume[z],cmap='Greys_r')
    ax[1].imshow(n_volume[:,x,:],cmap='Greys_r')
    ax[2].imshow(n_volume[:,:,y],cmap='Greys_r')
else:
    fig,axes=plt.subplots(1,3,figsize=(12,6),sharex=True,sharey=True)
    ax = axes.ravel()
    ax[0].set_title('Labelled slices')
    m=ax[0].imshow(labels,cmap=cmap,vmin=1)
    cb=fig.colorbar(m,ticks=range(nphases+1))
    m.set_clim(0.5, nphases + 0.5)
    ax[1].set_title('X-ray data')
    ax[1].imshow(x_volume[slicenr],cmap='Greys_r')
    ax[2].set_title('Neutron data')
    ax[2].imshow(n_volume[slicenr],cmap='Greys_r')
 

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …